In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from functools import partial

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizer, AutoTokenizer
from transformer_lens import HookedTransformer
import torch.nn.functional as F

In [3]:
from eap.graph import Graph
from eap.evaluate import evaluate_graph, evaluate_baseline
from eap.attribute import attribute 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def load_model(adapter_path, hf_model_name, translens_model_name, scratch_cache_dir = None):
    base_model = AutoModelForCausalLM.from_pretrained(hf_model_name, cache_dir=scratch_cache_dir)
    model_with_lora = PeftModel.from_pretrained(base_model, adapter_path)
    model_with_lora = model_with_lora.merge_and_unload()
    model = HookedTransformer.from_pretrained(model_name=translens_model_name, hf_model=model_with_lora, cache_dir=scratch_cache_dir)  

    model.cfg.use_split_qkv_input = True
    model.cfg.use_attn_result = True
    model.cfg.use_hook_mlp_in = True
    model.cfg.ungroup_grouped_query_attention = True
    return model

In [5]:
def collate_EAP(xs):
    clean, corrupted, labels = zip(*xs)
    clean = list(clean)
    corrupted = list(corrupted)
    return clean, corrupted, labels

class EAPDataset(Dataset):
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)

    def __len__(self):
        return len(self.df)
    
    def shuffle(self):
        self.df = self.df.sample(frac=1)

    def head(self, n: int):
        self.df = self.df.head(n)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        return row['clean'], row['corrupted'], row['label']
    
    def to_dataloader(self, batch_size: int):
        return DataLoader(self, batch_size=batch_size, collate_fn=collate_EAP)
    
def get_logit_positions(logits: torch.Tensor, input_length: torch.Tensor):
    batch_size = logits.size(0)
    idx = torch.arange(batch_size, device=logits.device)

    logits = logits[idx, input_length - 1]
    return logits

def kl_divergence(logits: torch.Tensor, clean_logits: torch.Tensor, input_length: torch.Tensor, labels: torch.Tensor, mean=True, loss=True):
    logits = get_logit_positions(logits, input_length)
    clean_logits = get_logit_positions(clean_logits, input_length)

    probs = torch.softmax(logits, dim=-1)
    clean_probs = torch.softmax(clean_logits, dim=-1)
    results = F.kl_div(probs.log(), clean_probs.log(), log_target=True, reduction='none').mean(-1)
    return results.mean() if mean else results

In [6]:
scratch_cache_dir = "/mnt/fast0/rje41/.cache/huggingface"    
hf_model_name = "EleutherAI/pythia-1.4B-deduped"
translens_model_name="pythia-1.4B-deduped"
adapter_path = "../fine-tuning/sequential-fine-tuning/adapter_checkpoints/add_sub_100/checkpoint-282"
model = load_model(
        adapter_path=adapter_path,
        hf_model_name=hf_model_name,
        translens_model_name=translens_model_name,
        scratch_cache_dir=scratch_cache_dir,
    )

Loaded pretrained model pythia-1.4B-deduped into HookedTransformer


In [13]:
ds = EAPDataset('../datasets/Add_Sub_100_circuit.csv')
dataloader = ds.to_dataloader(6)

In [14]:
g = Graph.from_model(model)
attribute(model, g, dataloader, partial(kl_divergence, loss=True, mean=True), method='EAP-IG-inputs', ig_steps=5)

100%|██████████| 84/84 [05:10<00:00,  3.70s/it]


In [ ]:
total_edges = len(g.edges)
five_percent_edges = int(total_edges * 0.05)
g.apply_topn(five_percent_edges , absolute=True)
g.to_json('graph.json')
g.edges

{'input': Node(input, in_graph: True),
 'a0.h0': Node(a0.h0, in_graph: True),
 'a0.h1': Node(a0.h1, in_graph: True),
 'a0.h2': Node(a0.h2, in_graph: True),
 'a0.h3': Node(a0.h3, in_graph: True),
 'a0.h4': Node(a0.h4, in_graph: True),
 'a0.h5': Node(a0.h5, in_graph: True),
 'a0.h6': Node(a0.h6, in_graph: True),
 'a0.h7': Node(a0.h7, in_graph: True),
 'a0.h8': Node(a0.h8, in_graph: True),
 'a0.h9': Node(a0.h9, in_graph: True),
 'a0.h10': Node(a0.h10, in_graph: True),
 'a0.h11': Node(a0.h11, in_graph: True),
 'a0.h12': Node(a0.h12, in_graph: True),
 'a0.h13': Node(a0.h13, in_graph: True),
 'a0.h14': Node(a0.h14, in_graph: True),
 'a0.h15': Node(a0.h15, in_graph: True),
 'm0': Node(m0, in_graph: True),
 'a1.h0': Node(a1.h0, in_graph: True),
 'a1.h1': Node(a1.h1, in_graph: True),
 'a1.h2': Node(a1.h2, in_graph: True),
 'a1.h3': Node(a1.h3, in_graph: True),
 'a1.h4': Node(a1.h4, in_graph: True),
 'a1.h5': Node(a1.h5, in_graph: True),
 'a1.h6': Node(a1.h6, in_graph: True),
 'a1.h7': Node(a1.h

In [16]:
def calculate_faithfulness(model, g, dataloader, metric_fn):
    baseline_performance = evaluate_baseline(model, dataloader, metric_fn).mean().item()
    circuit_performance = evaluate_graph(model, g, dataloader, metric_fn,skip_clean=False).mean().item()
    faithfulness = abs(baseline_performance - circuit_performance)
    percentage_performance = (1 - faithfulness / baseline_performance) * 100

    print(f"Baseline performance: {baseline_performance}")
    print(f"Circuit performance: {circuit_performance}")
    print(f"Faithfulness: {faithfulness}")
    print(f"Percentage of model performance achieved by the circuit: {percentage_performance:.2f}%")

    return faithfulness, percentage_performance

metric_fn = partial(kl_divergence, loss=False, mean=False)
faithfulness, percentage_performance = calculate_faithfulness(model, g, dataloader, metric_fn)

  1%|          | 1/84 [00:00<00:55,  1.49it/s]

here


  2%|▏         | 2/84 [00:01<00:55,  1.48it/s]

here


  4%|▎         | 3/84 [00:02<00:54,  1.49it/s]

...

here


 99%|█████████▉| 83/84 [00:55<00:00,  1.49it/s]

here


100%|██████████| 84/84 [00:55<00:00,  1.51it/s]

here
Baseline performance: 0.0003850016219075769
Circuit performance: 0.00032297518919222057
Faithfulness: 6.202643271535635e-05
Percentage of model performance achieved by the circuit: 83.89%
